# Autograd: 自动求导(automatic differentiation)
PyTorch中，所有神经网络的核心是torch.autograd包。autograd包为张量上的所有操作提供了自动求导机制。它是一个在运行时定义（define-by-run）的框架，这意味着反向传播是根据代码如何运行来决定的，并且每次迭代可以是不同的。    
```autograd.Function```实现一个自动求导操作的前向和反向定义, 每个张量操作都会创建至少一个Function节点，该节点连接到创建张量并对其历史进行编码的函数。  
  
  
## 1. tensor  
### 1.1 torch.Tensor
torch.Tensor是这个包的核心类。  
**如果设置它的属性 ```.requires_grad```为True，那么它将会追踪对于该张量的所有操作**。   
当完成计算后可以通过调用```.backward()```，来自动计算所有的梯度。这个张量的所有梯度将会自动累加到```.grad```属性。  
  
要阻止一个张量被跟踪历史，可以调用.detach()方法将其与计算历史分离，并阻止它未来的计算记录被跟踪。  
  
为了防止跟踪历史记录（和使用内存），可以将代码块包装在```with torch.no_grad():```中。在评估模型时特别有用，因为模型可能具有```requires_grad = True```的可训练的参数，但是我们不需要在此过程中对他们进行梯度计算。  
  
### 1.2 Function  
还有一个类对于autograd的实现非常重要：Function。

Tensor和Function互相连接生成了一个非循环图，它编码了完整的计算历史。每个```Tensor```都有一个```.grad_fn```属性，它引用了一个创建了这个```Tensor```的```Function```（除非这个张量是用户手动创建的，即这个张量的```grad_fn```是```None```）。

如果需要计算导数，可以在Tensor上调用```.backward()```。如果Tensor是一个标量（即它包含一个元素的数据），则不需要为backward()指定任何参数，但是如果它有更多的元素，则需要指定一个gradient参数，它是形状匹配的张量。

In [2]:
import torch

### 1.3 创建一个张量并设置requires_grad=True用来追踪其计算历史

In [4]:
x = torch.ones(2, 2, requires_grad=True)
print(x.dtype)  
y = x + 2
print(y)

torch.float32
tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


因为y是通过一个操作创建的,所以它有grad_fn,而x是由用户创建,所以它的grad_fn为None.

In [5]:
print(y.grad_fn)
print(x.grad_fn)

None


In [10]:
z = y * y * 3
out = z.mean()

print(z)
print(out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>)
tensor(27., grad_fn=<MeanBackward1>)


.requires\_grad_(...) 原地改变了现有张量的 requires_grad 标志。如果没有指定的话，默认输入的这个标志是False

In [12]:
a = torch.randn(2, 2)
a = ((a * 3) / (a - 1))
print(a)

print(a.requires_grad)
a.requires_grad_(True)
print(a.requires_grad)

b = (a * a).sum()
print(b)
print(b.grad_fn)

tensor([[ 117.7893,   -7.0627],
        [   1.1117, -325.9521]])
False
True
tensor(120170.2109, grad_fn=<SumBackward0>)


## 2. 梯度
因为out是一个标量。所以让我们直接进行反向传播，```out.backward()```和```out.backward(torch.tensor(1.))```等价

In [13]:
out.backward()

输出out对x的梯度```d(out)/dx```:

In [14]:
print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


You should have got a matrix of ``4.5``. Let’s call the ``out``
*Tensor* “$o$”.
We have that $o = \frac{1}{4}\sum_i z_i$,
$z_i = 3(x_i+2)^2$ and $z_i\bigr\rvert_{x_i=1} = 27$.
Therefore,
$\frac{\partial o}{\partial x_i} = \frac{3}{2}(x_i+2)$, hence
$\frac{\partial o}{\partial x_i}\bigr\rvert_{x_i=1} = \frac{9}{2} = 4.5$.


Mathematically, if you have a vector valued function $\vec{y}=f(\vec{x})$,
then the gradient of $\vec{y}$ with respect to $\vec{x}$
is a Jacobian matrix:

\begin{align}J=\left(\begin{array}{ccc}
   \frac{\partial y_{1}}{\partial x_{1}} & \cdots & \frac{\partial y_{m}}{\partial x_{1}}\\
   \vdots & \ddots & \vdots\\
   \frac{\partial y_{1}}{\partial x_{n}} & \cdots & \frac{\partial y_{m}}{\partial x_{n}}
   \end{array}\right)\end{align}

Generally speaking, ``torch.autograd`` is an engine for computing
Jacobian-vector product（是计算雅可比向量积的一个“引擎”）. That is, given any vector
$v=\left(\begin{array}{cccc} v_{1} & v_{2} & \cdots & v_{m}\end{array}\right)^{T}$,
compute the product $J\cdot v$. If $v$ happens to be
the gradient of a scalar function $l=g\left(\vec{y}\right)$,
that is,
$v=\left(\begin{array}{ccc}\frac{\partial l}{\partial y_{1}} & \cdots & \frac{\partial l}{\partial y_{m}}\end{array}\right)^{T}$,
then by the chain rule, the Jacobian-vector product would be the
gradient of $l$ with respect to $\vec{x}$:

\begin{align}J\cdot v=\left(\begin{array}{ccc}
   \frac{\partial y_{1}}{\partial x_{1}} & \cdots & \frac{\partial y_{m}}{\partial x_{1}}\\
   \vdots & \ddots & \vdots\\
   \frac{\partial y_{1}}{\partial x_{n}} & \cdots & \frac{\partial y_{m}}{\partial x_{n}}
   \end{array}\right)\left(\begin{array}{c}
   \frac{\partial l}{\partial y_{1}}\\
   \vdots\\
   \frac{\partial l}{\partial y_{m}}
   \end{array}\right)=\left(\begin{array}{c}
   \frac{\partial l}{\partial x_{1}}\\
   \vdots\\
   \frac{\partial l}{\partial x_{n}}
   \end{array}\right)\end{align}

This characteristic of Jacobian-vector product makes it very convenient to feed external gradients into a model that has non-scalar output.



现在我们来看一个雅可比向量积的例子:

In [24]:
'''
x = torch.randn(3, requires_grad=True)
y = x * 2
print(y)
# data.norm():P范数，p默认为2
while y.data.norm() < 1000:
    y = y * 2
    print(y)
'''
x = torch.randn(3, requires_grad=True)
y = torch.randn(3, requires_grad=True)
t = 2*x+y


 假设 x 经过一番计算得到 y，那么 y.backward(w) 求的不是 y 对 x 的导数，而是 l = torch.sum(y\*w) 对 x 的导数。w 可以视为 y 的各分量的权重，也可以视为遥远的损失函数 l 对 y 的偏导数。也就是说，不一定需要从计算图最后的节点 y 往前反向传播，从中间某个节点 n 开始传也可以，只要你能把损失函数 l 关于这个节点的导数 dl/dn 记录下来，n.backward(dl/dn) 照样能往前回传，正确地计算出损失函数 l 对于节点 n 之前的节点的导数。特别地，若 y 为标量，w 取默认值 1.0，才是按照我们通常理解的那样，求 y 对 x 的导数。

In [25]:
v = torch.tensor([0.1, 1.0, 0.0001], dtype=torch.float)
t.backward(v)
print(x.grad)

tensor([2.0000e-01, 2.0000e+00, 2.0000e-04])


In [13]:
x = torch.ones(2, 2, requires_grad=True)
print(x)
y = 3 * x + 2
print(y)
y.backward(torch.ones(2, 2))
print(x.grad.data)
print(type(x.grad.data))

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
tensor([[5., 5.],
        [5., 5.]], grad_fn=<AddBackward0>)
tensor([[3., 3.],
        [3., 3.]])
<class 'torch.Tensor'>
